### Imports

In [116]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F

### Récuperation des données qui sont dans le CSV

In [117]:
datas = pd.read_csv('datas/datas.csv', delimiter='\t')

### Récupération de la date

In [118]:
headers = datas.columns.drop('Time')

datas['Day'] = datas.Time.apply(lambda x: x.split('-')[-1])
datas['Month'] = datas.Time.apply(lambda x: x.split('-')[1])
datas['Year'] = datas.Time.apply(lambda x: x.split('-')[0])

In [119]:
datas

,Time,"BTC / Price, USD","ETH / Price, USD","XRP / Price, USD","LINK / Price, USD","XLM / Price, USD","USDT / Price, USD","USDT_ETH / Price, USD","USDT_TRX / Price, USD","DASH / Price, USD",...,"WTC / Price, USD","XEM / Price, USD","XMR / Price, USD","XVG / Price, USD","XTZ / Price, USD","YFI / Price, USD","ZEC / Price, USD",Day,Month,Year
0,2017-03-01,1228.459651,17.265684,0.005330,NaN,0.001731,0.999667,NaN,NaN,43.279388,...,NaN,0.007874,12.377001,NaN,NaN,NaN,41.565084,01,03,2017
1,2017-03-02,1262.931467,19.204561,0.006133,NaN,0.001799,1.006446,NaN,NaN,42.546546,...,NaN,0.008749,13.213293,NaN,NaN,NaN,40.045525,02,03,2017
2,2017-03-03,1289.363191,19.654703,0.006466,NaN,0.001789,1.003244,NaN,NaN,47.777786,...,NaN,0.009055,14.029060,NaN,NaN,NaN,42.348426,03,03,2017
3,2017-03-04,1268.940664,18.684225,0.006247,NaN,0.001750,1.005842,NaN,NaN,43.735475,...,NaN,0.009835,13.808267,NaN,NaN,NaN,41.372994,04,03,2017
4,2017-03-05,1276.273683,19.301111,0.006151,NaN,0.001742,0.999961,NaN,NaN,42.373144,...,NaN,0.011450,15.263604,NaN,NaN,NaN,39.567362,05,03,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807,2022-02-10,43614.342696,3080.565482,0.834418,17.553471,0.232654,1.000671,1.000671,1.000671,111.891064,...,0.593469,0.114365,180.655497,0.012299,4.152122,24498.512741,127.082193,10,02,2022
1808,2022-02-11,42357.321742,2924.080468,0.759247,16.146358,0.215302,1.000517,1.000517,1.000517,103.997657,...,0.550347,0.108235,168.596810,0.011535,4.220464,23068.136834,116.299055,11,02,2022
1809,2022-02-12,42173.459396,2910.836175,0.819350,15.943319,0.217710,1.000562,1.000562,1.000562,103.734638,...,0.552108,0.108695,173.587783,0.011503,4.089775,23100.542446,117.484142,12,02,2022
1810,2022-02-13,42195.594745,2880.696526,0.809239,15.718898,0.211419,1.000460,1.000460,1.000460,102.548106,...,0.546389,0.110236,174.332235,0.011014,3.871521,22934.195769,117.312547,13,02,2022


In [120]:
liste_crypto_df = {}
liste_title = []
for crypto in headers:
    df = pd.DataFrame(datas[crypto])

    title = crypto.split(' ')[0]
    liste_title.append(title)
    liste_crypto_df[title] = df

In [121]:
liste_crypto_df['ETH']

,"ETH / Price, USD"
0,17.265684
1,19.204561
2,19.654703
3,18.684225
4,19.301111
...,...
1807,3080.565482
1808,2924.080468
1809,2910.836175
1810,2880.696526


In [226]:
# Définir le Dataset
taille = 150

class CryptoDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe
        self.features = "Price"
        self.targets = dataframe.values  # 'ETH' column as target variable

    def __len__(self):
        return len(self.dataframe) - (taille + 1)

    def __getitem__(self, idx):
        features = torch.tensor(self.dataframe.iloc[idx : idx + taille].values, dtype=torch.float32)
        target = torch.tensor(self.targets[idx + taille], dtype=torch.float32)

        return features, target

# Utilisation du dataset
crypto_dataset = CryptoDataset(liste_crypto_df['ETH'])

In [227]:
print(len(crypto_dataset))

# Avant de passer les données à votre modèle, normalisez-les
mean = crypto_dataset.dataframe.mean()
std = crypto_dataset.dataframe.std()

crypto_dataset.dataframe = (crypto_dataset.dataframe - mean) / std

1661


In [234]:
# Définir le DataLoader
batch_size = 32  # Choisir la taille du lot en fonction de vos besoins
crypto_dataloader = DataLoader(crypto_dataset, 
                               batch_size=batch_size, 
                               shuffle=False)

In [235]:
class CryptoPredict(nn.Module):
    def __init__(self, input_size, hidden_size, num_stacked_layers):
        super(CryptoPredict, self).__init__()

        self.hidden_size = hidden_size
        self.num_stacked_layers = num_stacked_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_stacked_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.batch_norm = nn.BatchNorm1d(1)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        
        return out

# Usage of the model
input_size = 1
hidden_size = 40
num_stacked_layers = 1  # You may adjust this value

model = CryptoPredict(input_size, hidden_size, num_stacked_layers)

In [236]:
# Criterion and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

In [237]:
# Training loop
num_epoch = 800

for epoch in range(num_epoch): 
    running_loss = 0.0
    for i, data in enumerate(crypto_dataloader, 0):
        inputs, labels = data
        
        optimizer.zero_grad()

        outputs = model(inputs)

        #print("output : ", outputs)
        #print("labels : ", labels)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


        
        
        running_loss += loss.item()
            
    print(f'Epoch {epoch + 1}/{num_epoch}, Loss: {running_loss / len(crypto_dataloader)}')

Epoch 1/800, Loss: 2301140.2992412862
Epoch 2/800, Loss: 2295223.7459810697
Epoch 3/800, Loss: 2291310.5788010815
Epoch 4/800, Loss: 2288152.1503342846
Epoch 5/800, Loss: 2285239.8457594654
Epoch 6/800, Loss: 2282348.7649113582
Epoch 7/800, Loss: 2279524.7591458834
Epoch 8/800, Loss: 2276828.0467059794
Epoch 9/800, Loss: 2273951.452843299
Epoch 10/800, Loss: 2271153.3064434347
Epoch 11/800, Loss: 2268491.6288686898
Epoch 12/800, Loss: 2265885.2790527344
Epoch 13/800, Loss: 2263318.325063852
Epoch 14/800, Loss: 2260782.6066331128
Epoch 15/800, Loss: 2258273.0309213493
Epoch 16/800, Loss: 2255786.1378079928
Epoch 17/800, Loss: 2253319.4059213493
Epoch 18/800, Loss: 2250870.410888672
Epoch 19/800, Loss: 2248438.1301739034
Epoch 20/800, Loss: 2246021.049128606
Epoch 21/800, Loss: 2243618.1212252104
Epoch 22/800, Loss: 2241228.4814453125
Epoch 23/800, Loss: 2238851.3658259464
Epoch 24/800, Loss: 2236486.149104192
Epoch 25/800, Loss: 2234132.4513502857
Epoch 26/800, Loss: 2231789.6564847506


KeyboardInterrupt: 